In [109]:
import sys
import spikeinterface as si
import matplotlib.pyplot as plt
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import probeinterface

from probeinterface import Probe, ProbeGroup
from probeinterface.plotting import plot_probe, plot_probegroup
from probeinterface import generate_dummy_probe, generate_linear_probe
from probeinterface import write_probeinterface, read_probeinterface
from probeinterface import write_prb, read_prb
import os
import numpy as np
from spikeinterface.core import concatenate_recordings
import seaborn as sns
import spikeinterface.exporters as sexp


In [110]:
file_list = os.listdir(f"/media/ubuntu/sda/littlerat/raw_data/rhs/20250616littlerat_12_250616_170910")
file_list.remove("settings.xml")
recording_raw_list = []
for file in file_list:
    recording_raw_list.append(se.read_intan(f"/media/ubuntu/sda/littlerat/raw_data/rhs/20250616littlerat_12_250616_170910/{file}", stream_id= '0'))
recording_raw = concatenate_recordings(recording_list=recording_raw_list)
recording_recorded = recording_raw.remove_channels(['A-001', 'A-031'])

In [111]:
recording_f = spre.bandpass_filter(recording_recorded, freq_min=300, freq_max=750)
recording_f = spre.notch_filter(recording_f, freq=50)

In [112]:
#recording_f = recording_f.channel_slice(channel_ids=['A-011', 'A-012', 'A-015', 'A-024'])
recording_f = recording_f.channel_slice(channel_ids=['A-012', 'A-015'])

In [113]:
probe_position = np.array(((0, 0), (0, 200)))
#probe_position = np.array(((0, 0), (0, 200), (200, 0), (200, 200)))

In [114]:
probe_2d = Probe(ndim=2, si_units='um')
probe_2d.set_contacts(positions=probe_position, shapes='circle', shape_params={'radius': 20})
probe_2d.create_auto_shape()
probe_2d.set_device_channel_indices([0, 1])

In [115]:
recording_f = recording_f.set_probegroup(probe_2d)
#recording_cmr = spre.common_reference(recording_f, reference="", operator="median")


In [116]:
recording_preprocessed = recording_f.save(format="binary")

output_folder = '/media/ubuntu/sda/littlerat/test_littlerat'
sorting_kilosort4 = ss.run_sorter(sorter_name="kilosort4", recording=recording_preprocessed, output_folder=output_folder + "/kilosort4", 
                                  Th_universal = 8, Th_learned = 7, Th_single_ch = 5)
analyzer_kilosort4 = si.create_sorting_analyzer(sorting=sorting_kilosort4, recording=recording_preprocessed, format='binary_folder', folder=output_folder + '/analyzer_kilosort4_binary')

extensions_to_compute = [
    "random_spikes",
    "waveforms",
    "noise_levels",
    "templates",
    "spike_amplitudes",
    "unit_locations",
    "spike_locations",
    "correlograms",
    "template_similarity"
]

extension_params = {
    "unit_locations": {"method": "center_of_mass"},
    "spike_locations": {"ms_before": 0.1},
    "correlograms": {"bin_ms": 0.1},
    "template_similarity": {"method": "cosine_similarity"}
}

analyzer_kilosort4.compute(extensions_to_compute, extension_params=extension_params)

qm_params = sqm.get_default_qm_params()
analyzer_kilosort4.compute("quality_metrics", qm_params)

sexp.export_to_phy(analyzer_kilosort4, output_folder + "/phy_folder_for_kilosort", verbose=True)
    

Use cache_folder=/tmp/spikeinterface_cache/tmpwoqcskua/D6YN7M1G
write_binary_recording 
engine=process - n_jobs=1 - samples_per_chunk=20,000 - chunk_memory=78.12 KiB - total_memory=78.12 KiB - chunk_duration=1.00s


write_binary_recording (no parallelization):   0%|          | 0/1804 [00:00<?, ?it/s]

write_binary_recording (no parallelization): 100%|██████████| 1804/1804 [00:04<00:00, 417.08it/s]
/tmp/ipykernel_76086/365932873.py:4: DeprecationWarning: `output_folder` is deprecated and will be removed in version 0.103.0 Please use folder instead
  sorting_kilosort4 = ss.run_sorter(sorter_name="kilosort4", recording=recording_preprocessed, output_folder=output_folder + "/kilosort4",
/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.11/site-packages/kilosort/run_kilosort.py:488: UserWarning: 
            Parameter `nearest_chans` must be less than or equal to the number 
            of data channels being sorted.

            Changing from 10 to 2.
            
  warnings.warn(msg, UserWarning)
estimate_sparsity (no parallelization): 100%|██████████| 1804/1804 [00:00<00:00, 125161.68it/s]
/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.11/site-packages/spikeinterface/core/basesorting.py:261: UserWarning: The registered recording will not be persistent on disk, but only av

Run:
phy template-gui  /media/ubuntu/sda/littlerat/test_littlerat/phy_folder_for_kilosort/params.py
